In [ ]:
import json
from database_utils import Session; session = Session();


### Wait Time Data
Every row is represents a judicial appointment. It contains:
* All the judge's data
* Information about the appointment (court type)
* Congressional Start Year of the nomination.
* Nominating President
* congressional makeup

In [3]:

session.rollback()
wait_data = session.execute(
"""
SELECT
    j.*,
    con.start_year AS congress_start_year,
    appt.start_year AS appointent_year,
    appt.nomination_date,
    appt.days_to_confirm,
    appt.court_name,
    con.president_party_senate_majority_perc,
    con.president_party_senate_majority,
    con.party_of_president,
    con.president
FROM appointment appt
JOIN congress con
    ON date_part('year', appt.nomination_date) >= con.start_year
    AND date_part('year', appt.nomination_date) < con.end_year
JOIN judge j
    ON appt.nid = j.nid
WHERE appt.days_to_confirm IS NOT NULL
"""
)
wait_data_export = []
for row in wait_data:
    row = dict(row)
    row['nomination_date'] = str(row['nomination_date'])
    wait_data_export.append(row)
json.dump(wait_data_export, open('./joined_judges_wait.json', 'w'))

### Education

### Unsuccessful Nominations

Taken from another dataset, so unique identifers like JID and NID are not present

Every row is represents a judicial nomination that was unsuccessful . It contains:

* Nominating President
* Congress of nomination year
* Judge Name
* Court Name
* Nomination Date
* Outcome



In [4]:
unsuccesful_nominations = session.execute(
"""
SELECT * FROM unsuccessful_nomination
""")
unsuccesful_nominations = [dict(row) for row in unsuccesful_nominations]
json.dump(wait_data_export, open('./unsuccessful_nominations.json', 'w'))